In [1]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
import matplotlib.pyplot as plt
import zipfile
import requests
!pip install torchinfo
from torchinfo import summary
!git clone 'https://github.com/mrdbourke/pytorch-deep-learning'
dir_path = 'functions'
!mkdir {dir_path}
!mv pytorch-deep-learning/going_modular functions
!rm -rf pytorch-deep-learning
!ls

Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4393, done.
remote: Total 4393 (delta 0), reused 0 (delta 0), pack-reused 4393 (from 1)
Receiving objects: 100% (4393/4393), 764.14 MiB | 15.97 MiB/s, done.
Resolving deltas: 100% (2656/2656), done.
Updating files: 100% (248/248), done.
,  sample_data


In [4]:
from functions.going_modular import data_setup,engine

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
import os
from pathlib import Path

In [9]:
data_path = Path('data/')

data_path.mkdir(parents=True,exist_ok=True)
request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip')

with open('pizza_steak_sushi_20_percent.zip','wb') as f:
  f.write(request.content)

with zipfile.ZipFile('pizza_steak_sushi_20_percent.zip','r') as f:
  f.extractall(data_path)

In [10]:
train_dir = data_path / 'train'
test_dir = data_path / 'test'

In [12]:
manual_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [14]:
train_dataloader,test_dataloader,class_names = data_setup.create_dataloaders(train_dir,test_dir,manual_transform,batch_size=32)

In [16]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
#tranform = weights.transforms()

In [17]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 196MB/s]


In [20]:
summary(model=model,input_size=(32,3,224,224),col_names=['input_size','output_size','num_params','trainable'],row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape               Output Shape              Param #                   Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]         [32, 1000]                --                        True
├─Sequential (features)                                      [32, 3, 224, 224]         [32, 1280, 7, 7]          --                        True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]         [32, 32, 112, 112]        --                        True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]         [32, 32, 112, 112]        864                       True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]        [32, 32, 112, 112]        64                        True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]        [32, 32, 112, 112]        --                

In [21]:
for param in model.features.parameters():
  param.requires_grad = False

In [23]:
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=1280,out_features=3,bias=True)
).to(device)

In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(params=model.parameters(),lr=0.001)

results = engine.train(model=model,train_dataloader=train_dataloader,test_dataloader=test_dataloader,optimizer=optimizer,loss_fn=loss_fn,epochs=5,device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9982 | train_acc: 0.5104 | test_loss: 0.6914 | test_acc: 0.8881
Epoch: 2 | train_loss: 0.7175 | train_acc: 0.8125 | test_loss: 0.5441 | test_acc: 0.8977
Epoch: 3 | train_loss: 0.5893 | train_acc: 0.8417 | test_loss: 0.4737 | test_acc: 0.8943
Epoch: 4 | train_loss: 0.4488 | train_acc: 0.9292 | test_loss: 0.4410 | test_acc: 0.9131
Epoch: 5 | train_loss: 0.4299 | train_acc: 0.8583 | test_loss: 0.3802 | test_acc: 0.9193
